In [23]:
import re
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from IPython.display import display, Markdown
from langchain.storage import InMemoryByteStore
from langchain.retrievers import ParentDocumentRetriever
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [24]:
load_dotenv()

True

In [25]:
groq_api_key = os.getenv("GROQ_API_KEY")
print(groq_api_key[:5])

gsk_i


In [26]:
with open("corpus.txt", "r", encoding="utf-8") as f:
    corpus = f.read()

print(corpus[:200])

### MODELOS DISPONIBLES

Modelo: FZ-S FI
Cilindrada: 149 cc
Precio: $3.500.000
Descripción: Con estilo deportivo y tecnología Blue Core, el FZ-S FI ofrece eficiencia y rendimiento para el uso diario e


In [27]:
# Segmentamos el corpus en bloques útiles (por ejemplo, por modelo o por FAQ)
segments = re.split(r"\n\s*---+\s*\n|\n\s*### ", corpus)
segments = [c.strip() for c in segments if c.strip()]

# Creamos los objetos Document
documents = [Document(page_content=chunk) for chunk in segments]

In [28]:
len(documents)

4

In [29]:
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
embedder = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [30]:
# Creación de la base de datos vectorial y almacenamiento en memoria
vectordb = Chroma(
	collection_name="parent_child_demo",
	embedding_function=embedder,
	persist_directory="../src/vectordb/"
)

In [31]:
byte_store = InMemoryByteStore()

In [32]:
retriever = ParentDocumentRetriever(
	vectorstore=vectordb,
	byte_store=byte_store,
	child_splitter=splitter
)

In [33]:
retriever.add_documents(documents)

In [34]:
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama3-8b-8192",
    temperature=0.2,
	max_tokens=100
)

In [35]:
template = """
Sos un asistente de atención al cliente para la concesionaria de motos LiderMotos. Te caracterizás por ser conciso y directo, entendiendo que los usuarios interactúan con nosotros a través de WhatsApp y prefieren respuestas breves y claras. Pero, no te olvides siempre de saludar y ser amable. Tu objetivo es proporcionar respuestas útiles y precisas a las preguntas de los usuarios, basándote en la información disponible en el contexto proporcionado.

Contexto:
{contexto}

Pregunta: {pregunta}
Respuesta:
"""

prompt = PromptTemplate(
    input_variables=["contexto", "pregunta"],
    template=template
)

In [36]:
query = "¿Puedo pagar en cuotas con tarjeta naranja?"

In [37]:
resultados = retriever.get_relevant_documents(query)
contexto = "\n\n".join([doc.page_content for doc in resultados])

prompt_input = prompt.format(contexto=contexto, pregunta=query)

In [38]:
# Llamada al modelo Groq
respuesta = llm.invoke(prompt_input)
display(Markdown(respuesta.content))

¡Hola!